In [56]:
import pandas as pd
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [75]:
data = pd.read_csv('tweets-data/fufufafa.csv')
data.head()

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1833186816733942051,Mon Sep 09 23:59:40 +0000 2024,5,@mas_veel @kafiradikalis Lho org iba pd prabow...,1833294230049194002,NaN,mas_veel,in,NaN,0,2,0,https://x.com/ucieamoi/status/1833294230049194002,1788852989027090432,ucieamoi
1,1833294181537878397,Mon Sep 09 23:59:28 +0000 2024,0,Tanpa fufufafa pun milih mereka udah salah.......,1833294181537878397,NaN,NaN,in,NaN,0,0,0,https://x.com/briizenriize/status/183329418153...,1031705095,briizenriize
2,1833294132489687389,Mon Sep 09 23:59:16 +0000 2024,1,Sama kayak fufufafa yg nebeng sama Wowo biar j...,1833294132489687389,NaN,NaN,in,Jakarta sebelah kanan,0,0,0,https://x.com/NoKunyuk_Istana/status/183329413...,230040721,NoKunyuk_Istana
3,1833133899989631301,Mon Sep 09 23:59:11 +0000 2024,0,@lordalwayslord @RheinaHerlina @Heraloebss Sem...,1833294111757242748,NaN,lordalwayslord,in,NaN,0,0,0,https://x.com/kosongualu/status/18332941117572...,1396204417431474176,kosongualu
4,1833139598492635554,Mon Sep 09 23:58:53 +0000 2024,0,@nikmatilahh @VIVAcoid Anak Abah lagi fokus no...,1833294036217803235,NaN,nikmatilahh,in,NaN,0,1,0,https://x.com/Kusumayani28698/status/183329403...,1768305302062215168,Kusumayani28698


In [76]:
data = data.drop(['id_str', 'conversation_id_str', 'image_url', 'tweet_url', 'user_id_str', 'username', 'in_reply_to_screen_name','quote_count',	'reply_count',	'retweet_count','favorite_count'], axis=1)


In [77]:
data.head()

,created_at,full_text,lang,location
0,Mon Sep 09 23:59:40 +0000 2024,@mas_veel @kafiradikalis Lho org iba pd prabow...,in,NaN
1,Mon Sep 09 23:59:28 +0000 2024,Tanpa fufufafa pun milih mereka udah salah.......,in,NaN
2,Mon Sep 09 23:59:16 +0000 2024,Sama kayak fufufafa yg nebeng sama Wowo biar j...,in,Jakarta sebelah kanan
3,Mon Sep 09 23:59:11 +0000 2024,@lordalwayslord @RheinaHerlina @Heraloebss Sem...,in,NaN
4,Mon Sep 09 23:58:53 +0000 2024,@nikmatilahh @VIVAcoid Anak Abah lagi fokus no...,in,NaN


In [60]:
variations=data['lang'].unique()
print(variations)

['in']


In [78]:
data = data.drop(['lang'], axis=1)

In [79]:
text = data

def extract_mentions(text):
    mentions = re.findall(r'@\w+', text)
    return mentions

text['mentions'] = text['full_text'].apply(extract_mentions)

mentions = text['mentions'].sum()
mention_count = Counter(mentions)
for mention, count in (mention_count.most_common(6)):
    print(f"{mention:<20} | {count:>5} times")


@prabowo             |    47 times
@gibran_tweet        |    23 times
@DokterTifa          |    18 times
@tekarok007          |    16 times
@Gerindra            |    14 times
@Chilli_Pari         |    11 times


In [80]:
def clean_text(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data['full_text'] = data['full_text'].apply(clean_text)

In [81]:
data['full_text'].head()

0    Lho org iba pd prabowo kok dibilang ngadu domb...
1    Tanpa fufufafa pun milih mereka udah salah unt...
2    Sama kayak fufufafa yg nebeng sama Wowo biar j...
3    Semangat pokoknya jangan sampe redup Kalo bisa...
4    Anak Abah lagi fokus nonton serial fufufafa se...
Name: full_text, dtype: object

In [82]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\calys\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\calys\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [83]:
#tokeniza
data['tokens'] = data['full_text'].apply(word_tokenize)

#stopwords
stop_words = set(stopwords.words('indonesian'))
data['tokens'] = data['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

#stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

data['stemmed_tokens'] = data['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

#join
data['clean_text'] = data['stemmed_tokens'].apply(lambda x: ' '.join(x))

In [85]:
data.head()

,created_at,full_text,location,mentions,tokens,stemmed_tokens,clean_text
0,Mon Sep 09 23:59:40 +0000 2024,Lho org iba pd prabowo kok dibilang ngadu domb...,NaN,"[@mas_veel, @kafiradikalis]","[Lho, org, iba, pd, prabowo, dibilang, ngadu, ...","[lho, org, iba, pd, prabowo, bilang, ngadu, do...",lho org iba pd prabowo bilang ngadu domba ting...
1,Mon Sep 09 23:59:28 +0000 2024,Tanpa fufufafa pun milih mereka udah salah unt...,NaN,[],"[fufufafa, milih, udah, salah, untung, gw, otak]","[fufufafa, milih, udah, salah, untung, gw, otak]",fufufafa milih udah salah untung gw otak
2,Mon Sep 09 23:59:16 +0000 2024,Sama kayak fufufafa yg nebeng sama Wowo biar j...,Jakarta sebelah kanan,[],"[kayak, fufufafa, yg, nebeng, Wowo, biar, wapr...","[kayak, fufufafa, yg, nebeng, wowo, biar, wapr...",kayak fufufafa yg nebeng wowo biar wapres sisa...
3,Mon Sep 09 23:59:11 +0000 2024,Semangat pokoknya jangan sampe redup Kalo bisa...,NaN,"[@lordalwayslord, @RheinaHerlina, @Heraloebss]","[Semangat, pokoknya, sampe, redup, Kalo, kawal...","[semangat, pokok, sampe, redup, kalo, kawal, f...",semangat pokok sampe redup kalo kawal fufufafa...
4,Mon Sep 09 23:58:53 +0000 2024,Anak Abah lagi fokus nonton serial fufufafa se...,NaN,"[@nikmatilahh, @VIVAcoid]","[Anak, Abah, fokus, nonton, serial, fufufafa, ...","[anak, abah, fokus, nonton, serial, fufufafa, ...",anak abah fokus nonton serial fufufafa seru ba...


In [86]:
data.to_csv('tweets-data/cleaned.csv',index=False)